In [89]:
!pip install --upgrade langchain langchain_community transformers accelerate python-dotenv langgraph
!pip install --upgrade langchain_huggingface


In [90]:

from typing import TypedDict
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langgraph.graph import StateGraph,START,END
from dotenv import load_dotenv

In [91]:

import os
from dotenv import load_dotenv


os.environ['HUGGING_FACE_HUB_TOKEN'] = ''


False

In [92]:


load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    temperature=0.1,
    repetition_penalty=1.1,
    timeout=120,
)

model = ChatHuggingFace(llm=llm)


In [93]:
#state
class States(TypedDict):
    Question:str
    Answer:str
    joke:str

In [94]:
def cht_mod(state:States) ->States:
    Q=state.get('Question')

    ans=model.invoke(Q)

    state['Answer']=ans.content

    return state

In [121]:
def joke(state:States) ->States:
    Q=state.get('Answer')

    joke=model.invoke(f"tell a joke about this context{Q}")

    state['joke']=joke.content

    return state


In [122]:
Graph=StateGraph(States)

Graph.add_node('llm',cht_mod)
Graph.add_node("joke",joke)

Graph.add_edge(START,'llm')
Graph.add_edge('llm','joke')
Graph.add_edge('joke',END)


In [123]:
workflow=Graph.compile()

In [124]:
question={'Question':"who is teh strongest avenger"}

In [125]:
final_state = workflow.invoke(question)

In [126]:
print(final_state)

{'Question': 'who is teh strongest avenger', 'Answer': 'According to comics and movies, Thor is often considered the strongest Avenger. However, Hulk and Captain Marvel also have incredible strength.\n', 'joke': 'The joke, "The strong Avenger – Thor, Captain Marvel, Hulk – but, ants!? Don\'t call him \'Ant-Problem\'!" is a play on words, comparing the powerful Avengers to ants, implying they are only problematic as insects and not as superheroes.\n'}
